In [1]:
# R Path - required by rpy2 prior to importing libraries
import os
import os
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"
os.environ["LD_LIBRARY_PATH"] = "/Library/Frameworks/R.framework/Resources/lib/"

# Basics
import numpy as np
import pandas as pd
from datetime import datetime

# Data
from pandas_datareader.data import DataReader

# R
from rpy2.robjects import pandas2ri

# Kalman Filter
from pykalman import KalmanFilter

# Machine Learning - tensorflow, keras, and sklearn
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Flatten, Lambda, MaxPooling3D, Conv3D, RepeatVector, Layer

# SKLearn Models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

# Statsmodels
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller

# Univariate GARCH

# Plots
from matplotlib import pyplot as plt
import seaborn as sns

# Misc
import warnings

from sklearn.feature_selection import RFE, mutual_info_regression
from sklearn.linear_model import LinearRegression



In [ ]:
#####################################################################################################################
#                                                                                                                   #
# Constants and Parameters                                                                                          #
#                                                                                                                   #
#####################################################################################################################

# Folders and worksheet names
str_Dir_Plan_FRED = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/FRED/'
str_Dir_Plan_Data = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PX/'
str_Dir_Plan_PC = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/PXNew/'
str_Dir_Results = '/Users/bodamjerry/Desktop/Winter2425/Applied Economics/data/Results/'

str_Nome_Plan_FRED_MD = 'current'
# str_Nome_Plan_FRED_MD = '2015-01'
str_Nome_Plan_FRED_MD_Desc = 'Data_Description_MD'

# How to display plots
%matplotlib inline 
plt.rcParams['figure.dpi'] = 200 # Plot resolution (dpi)

# Required to convert datatypes from Python to R and vice-versa
pandas2ri.activate()

# Remove warnings
warnings.filterwarnings('ignore')

# Color style (plots)
sns.set(color_codes = True)

# Statistical significance for hypothesis testing
# Using 1% due to the high number of tests carried out
alfa = 0.01

# Test size (share of observations used to build the test sample)
share_test_size = 0.20

# Validation sample size (share of observations used to build the validation sample)
share_validation_size = 0.20

# Number of lags considered when splitting the data - see LSTM models
n_lags_lstm = 2

# Number of lags considered when splitting the data - see ConvLSTM models
n_lags_conv = 2

# Number of sequences into which sample are broken when fitting ConvLSTM
# Note: n_lags = n_seq * n_steps
n_seq_conv = 1

# Size of each sequence into which sample are broken when fitting ConvLSTM
# Note: n_lags = n_seq * n_steps
n_steps_conv = int(n_lags_conv / n_seq_conv)

# Activation function
act_fun = 'selu'


In [ ]:
#####################################################################################################################
#                                                                                                                   #
# Auxiliary Functions                                                                                               #
#                                                                                                                   #
#####################################################################################################################

# Split a univariate sequence into samples
def split_sequence_uni(sequence, n_steps, per_ahead, cum = False):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix + per_ahead - 1 > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        if cum == False:
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + per_ahead - 1]
        else:
            seq_x, seq_y = sequence[i:end_ix], np.sum(sequence[end_ix:(end_ix + per_ahead)])
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Split a multivariate sequence into samples
def split_sequence_mult(sequences, n_steps, per_ahead, cum = False):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix + per_ahead - 1 > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        if cum == False:
            seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix + per_ahead - 1, -1]
        else:
            seq_x, seq_y = sequences[i:end_ix, :-1], np.sum(sequences[end_ix:(end_ix + per_ahead), -1])
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Kalman filter regression
# If EM = True, then EM algorithm is used for estimation
# delta is related to the variance of the betas. Delta -> 1 makes betas more volatile, which may lead to overfitting.
# However, delta -> 0 may increase the MSE.

def KFReg(X, y, delta, obs_cov, init_mean, init_cov, EM = False):
    n_features = X.shape[1]
    obs_mat = X[:, np.newaxis, :]
    if EM == False:
        trans_cov = (delta/(1 - delta))*np.eye(n_features)
        kf = KalmanFilter(n_dim_obs = 1, n_dim_state = n_features,
                          initial_state_mean = init_mean,
                          initial_state_covariance = init_cov,
                          transition_matrices = np.eye(n_features),
                          observation_matrices = obs_mat,
                          observation_covariance = obs_cov,
                          transition_covariance = trans_cov)
        state_means, state_covs = kf.filter(y)
    else:
        kf = KalmanFilter(n_dim_obs = 1, n_dim_state = n_features,
                          initial_state_mean = init_mean,
                          initial_state_covariance = init_cov,
                          observation_matrices = obs_mat)
    state_means, state_covs = kf.em(y).filter(y)
    return state_means, state_covs, kf

# Mean Absolute Error
def MAE(y_obs, y_hat):
    return np.mean(np.abs(y_obs - y_hat))

# Mean Squared Error
def MSE(y_obs, y_hat):
    return np.mean((y_obs - y_hat)**2)

# RMSE
def RMSE(y_obs, y_hat):
    return np.sqrt(MSE(y_obs, y_hat))

def MAPE(y_obs, y_hat):
    return np.mean(np.abs(y_obs - y_hat)/y_obs)

def cos_sim(y_obs, y_hat):
    return np.dot(y_obs, y_hat)/(np.linalg.norm(y_obs)*np.linalg.norm(y_hat))

def R2(y_obs, y_hat):
    SSR = np.sum((y_obs - y_hat)**2)
    SST = np.sum((y_obs - np.mean(y_obs))**2)
    return (1 - SSR/SST)

# Variational autoencoder
# Use those parameters to sample new points from the latent space:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
# z_mean = []
# z_log_var = []

def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Variational autoencoder

def vae(X_train, X_test, intermediate_dim, latent_dim, batch_size, epochs, verbose, plot_name):

    original_dim = X_train.shape[1]
    input_shape = (original_dim, )

    # Map inputs to the latent distribution parameters:
    # VAE model = encoder + decoder
    # build encoder model
    inputs = Input(shape=input_shape, name='encoder_input')

    print("Encoder Input Shape:", inputs.shape)

    x = Dense(intermediate_dim, activation='relu', name='intermediate_encoding')(inputs)
    print("Intermediate Layer Shape:", x.shape)

    z_mean = Dense(latent_dim, name='z_mean')(x)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)

    print("z_mean Shape:", z_mean.shape)
    print("z_log_var Shape:", z_log_var.shape)

    # use reparameterization trick to push the sampling out as input
    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

    # z = SamplingLayer(name='z')([z_mean, z_log_var])
    print("Sampling Layer Output Shape:", z.shape)

    # Instantiate the encoder model:
    # encoder = Model(inputs, z_mean)
    encoder = Model(inputs=inputs, outputs=[z_mean, z_log_var, z], name='encoder')
    #encoder = Model(inputs=inputs, outputs=z, name='encoder')
    encoder.summary()

    # Build the decoder model:
    print("LATENT DIMENSION:", latent_dim)
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    print("Decoder Input Shape:", latent_inputs.shape)

    x = Dense(intermediate_dim, activation='relu', name='intermediate_decoding')(latent_inputs)
    outputs = Dense(original_dim, activation='sigmoid')(x)

    print("Decoder Output Shape:", outputs.shape)

    # Instantiate the decoder model:
    decoder = Model(latent_inputs, outputs, name='decoder')
    decoder.summary()
    if latent_dim == 1:
        print("GOT HERE 1")

    # Instantiate the VAE model:
    outputs = decoder(encoder(inputs)[2])

    # outputs = decoder(encoder(inputs))
    vae = Model(inputs, outputs, name='vae_mlp')
    if latent_dim == 1:
        print("GOT HERE 2")


    # Define the VAE loss function
    def vae_loss(inputs, outputs, z_mean, z_log_var, original_dim):
        bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        reconstruction_loss = bce_loss(inputs, outputs) * original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        return K.mean(reconstruction_loss + kl_loss)


    # Add the loss to the model
    # vae.add_loss(vae_loss(inputs, outputs, z_mean, z_log_var, original_dim))
    # Compile the model
    if latent_dim == 1:
        print("GOT HERE 3")
    loss_output = Lambda(lambda x: vae_loss(*x, original_dim))([inputs, outputs, z_mean, z_log_var])
    vae = Model(inputs=inputs, outputs=[outputs, loss_output])  # Include loss in the outputs
    vae.compile(optimizer='rmsprop', loss=['mse', None])  # Use 'mse' for the first output (reconstruction), None for the second (loss)

    # vae.compile(optimizer='rmsprop', loss=None)
    if latent_dim == 1:
        print("GOT HERE 4")
    # Finally, we train the model:
    tf.config.run_functions_eagerly(True)
    results = vae.fit(X_train, X_train,
                      shuffle=True,
                      epochs=epochs,
                      batch_size=batch_size,
                      validation_split=0.1,
                      verbose=verbose)
    tf.config.run_functions_eagerly(False)

    '''
    # Plot training & validation loss values
    plt.plot(results.history['loss'])
    plt.plot(results.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.savefig(plot_name + '_loss')
    plt.close()
    '''

    # Use the encoded layer to encode the training input
    if latent_dim == 1:
        print("GOT HERE 5")
    encoded_data = encoder.predict(X_train)[2]
    print(encoded_data)
    encoded_data = pd.DataFrame(data = encoded_data,
                                index = X_train.index,
                                columns = ["PC_vae" + str(i) for i in np.arange(0, latent_dim)])

    '''
    # Plots encoded data
    sns.lineplot(data = encoded_data)
    plt.savefig(plot_name + '_PC')
    plt.close()
    '''

    # Correlation matrix
    print("Encoded data Correlation Matrix")
    print(encoded_data.corr())

    # Encoded data
    if latent_dim == 1:
        print("GOT HERE 5")
    X_train_encoded_vae = encoded_data
    encoded_data = encoder.predict(X_test)[2]
    encoded_data = pd.DataFrame(data = encoded_data,
                                index = X_test.index,
                                columns = ["PC_vae" + str(i) for i in np.arange(0, latent_dim)])
    X_test_encoded_vae = encoded_data

    X_train_encoded_vae.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_train.csv')
    X_test_encoded_vae.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_test.csv')
    if latent_dim == 1:
        print("GOT HERE 6")

    return X_train_encoded_vae, X_test_encoded_vae

def deep_ae(X_train, X_test, intermediate_dim, latent_dim, batch_size, epochs, verbose, plot_name):

    # Number of time series
    input_dim = X_train.shape[1]

    # if np.isnan(X_train).any() or np.isinf(X_train).any():
    #     print("Warning: X_train contains NaNs or Infs. Please check your data.")
    # if np.isnan(X_test).any() or np.isinf(X_test).any():
    #     print("Warning: X_test contains NaNs or Infs. Please check your data.")
    # 
    # Dimension of encoding units (roughly equivalent to principal components)
    encoding_dim1 = intermediate_dim
    encoding_dim2 = latent_dim

    # Autoencoder architecture
    input_img = Input(shape=(input_dim,), name = 'encoder_input')
    encoded_partial = Dense(encoding_dim1, activation = "selu", name = 'intermediate_encoding')(input_img)
    encoded = Dense(encoding_dim2, activation="selu", name = 'encoding_layer')(encoded_partial)
    decoded_partial = Dense(encoding_dim1, activation="selu", name = 'intermediate_decoding')(encoded)
    decoded = Dense(input_dim, activation="selu", name = 'decoding_layer')(decoded_partial)
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    print(autoencoder.summary())

    # Fits the autoencoder
    hist_autoencoder = autoencoder.fit(X_train, X_train,
                                       epochs=epochs,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       validation_split=0.1,
                                       verbose=verbose)

    # Use the encoded layer to encode the training input
    encoder = Model(input_img, encoded)
    encoded_input = Input(shape=(encoding_dim1,))
    decoder_layer = autoencoder.layers[-1]
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    encoded_data = encoder.predict(X_train)

    '''
    # Plots loss function
    plt.plot(hist_autoencoder.history['loss'])
    plt.plot(hist_autoencoder.history['val_loss'])
    plt.title('Model Train vs. Validation Loss (MSE)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.savefig(plot_name + '_loss')
    plt.close()
    '''

    # Converts encoded data to a labeled dataframe
    encoded_data = pd.DataFrame(data = encoded_data,
                                index = X_train.index,
                                columns = ["PC_ae" + str(i) for i in np.arange(0, encoding_dim2)])

    '''
    # Plots encoded data
    sns.lineplot(data = encoded_data)
    plt.savefig(plot_name + '_PC')
    plt.close()
    '''

    # Correlation matrix
    encoded_data.corr()

    # Stores the encoded data
    X_train_encoded_ae = encoded_data
    encoded_data = encoder.predict(X_test)
    encoded_data = pd.DataFrame(data = encoded_data,
                                index = X_test.index,
                                columns = ["PC_ae" + str(i) for i in np.arange(0, encoding_dim2)])
    X_test_encoded_ae = encoded_data

    X_train_encoded_ae.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_train.csv')
    X_test_encoded_ae.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_test.csv')

    return X_train_encoded_ae, X_test_encoded_ae

def pca_decomp(X_train, X_test, threshold, plot_name):

    # Runs PCA for the maximum number of components possible
    n_series = X_train.shape[1]
    pca = PCA(n_components = n_series, svd_solver = 'full')
    pca.fit(X_train)

    # Selects the number of PCs required for explained variance > threshold
    total_var = 0
    n_comp = 0
    for var in pca.explained_variance_ratio_:
        total_var = var + total_var
        n_comp = n_comp + 1
        if total_var > threshold:
            break

    # Runs PCA for the number of components selected
    pca = PCA(n_components = n_comp, svd_solver = 'full')
    pca.fit(X_train)

    # Applies transformation to training data
    X_train_pca = pca.transform(X_train)
    X_train_pca = X_train_pca.reshape(X_train_pca.shape[0], n_comp)
    X_train_pca = pd.DataFrame(data = X_train_pca,
                               index = X_train.index,
                               columns = ["PC" + str(i) for i in np.arange(0,n_comp)])

    # Applies transformation to test data
    X_test_pca = pca.transform(X_test)
    X_test_pca = X_test_pca.reshape(X_test_pca.shape[0], n_comp)
    X_test_pca = pd.DataFrame(data = X_test_pca,
                              index = X_test.index,
                              columns = ["PC" + str(i) for i in np.arange(0,n_comp)])

    '''
    # Plots training data
    sns.lineplot(data = X_train_pca)
    plt.savefig(plot_name + '_PC')
    plt.close()
    '''

    # Correlation matrix
    X_train_pca.corr()

    # Saves results
    X_train_pca.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_train.csv')
    X_test_pca.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + plot_name + '_test.csv')

    return X_train_pca, X_test_pca



In [ ]:
#####################################################################################################################
#                                                                                                                   #
# Data                                                                                                              #
#                                                                                                                   #
#####################################################################################################################

df_FRED_MD = pd.read_csv(filepath_or_buffer = str_Dir_Plan_FRED + str_Nome_Plan_FRED_MD + '.csv', skiprows=[1])
# df_FRED_MD.index = pd.to_datetime(df_FRED_MD.iloc[:,0])
df_date = df_FRED_MD['Date']

# df_FRED_MD = df_FRED_MD.drop(columns = 'Date')
df_FRED_MD['Date'] = pd.to_datetime(df_FRED_MD['Date'], format='%m/%d/%y', errors='coerce')
# Convert years from 2059-2099 to 1959-1999
df_FRED_MD.loc[df_FRED_MD['Date'].dt.year > 2024, 'Date'] -= pd.DateOffset(years=100)
df_FRED_MD.set_index('Date', inplace=True)

# print(df_FRED_MD.dropna())

print(df_FRED_MD.head(5))
print(df_FRED_MD.tail(5))

df_FRED_Desc_MD = pd.read_csv(filepath_or_buffer = str_Dir_Plan_FRED + str_Nome_Plan_FRED_MD_Desc + '.csv', sep = ';')
df_FRED_Desc_MD = df_FRED_Desc_MD.drop(columns = 'Index')



In [ ]:


def plot_correlation_matrices(df_desc, df_data, group_column, series_column):
    """
    Plots correlation matrices for each group of time series data.

    Args:
      df_desc: DataFrame containing series descriptions and group names.
      df_data: DataFrame containing the time series data.
      group_column: Name of the column with group labels in df_desc.
      series_column: Name of the column with series names in df_desc.
    """

    # Get unique group names
    group_names = df_desc[group_column].unique()

    for group_name in group_names:
        # Filter series for the current group
        series_names = df_desc[df_desc[group_column] == group_name][series_column]

        try:
            # Select corresponding data from df_data
            group_data = df_data[series_names]

            # Calculate the correlation matrix
            corr_matrix = group_data.corr()

            # Plot the correlation matrix
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
            plt.title(f'Correlation Matrix - {group_name}')
            plt.show()

        except KeyError as e:
            print(f"Warning: Series not found - {e}")


# Assuming your DataFrames are named df_FRED_Desc_MD and df_FRED_MD
plot_correlation_matrices(df_FRED_Desc_MD, df_FRED_MD, 'Group', 'FRED')


def select_highly_correlated_variables(df_desc, df_data, group_column, series_column, corr_threshold):
    """
    Selects highly correlated variables within each group of time series data.
    Handles KeyError if a series is not found in the data.

    Args:
      df_desc: DataFrame containing series descriptions and group names.
      df_data: DataFrame containing the time series data.
      group_column: Name of the column with group labels in df_desc.
      series_column: Name of the column with series names in df_desc.
      corr_threshold: Absolute correlation threshold for selecting variables.

    Returns:
      A dictionary where keys are group names and values are lists of 
      highly correlated variable pairs.
    """

    highly_correlated_vars = {}
    group_names = df_desc[group_column].unique()

    for group_name in group_names:
        highly_correlated_pairs = []  # Initialize here

        try:
            series_names = df_desc[df_desc[group_column] == group_name][series_column]
            group_data = df_data[series_names]
            corr_matrix = group_data.corr()

            for i in range(len(corr_matrix.columns)):
                for j in range(i + 1, len(corr_matrix.columns)):
                    var1 = corr_matrix.columns[i]
                    var2 = corr_matrix.columns[j]
                    corr_value = abs(corr_matrix.iloc[i, j])
                    if corr_value >= corr_threshold:
                        highly_correlated_pairs.append((var1, var2, corr_value))

        except KeyError as e:
            print(f"Warning: Series not found in group {group_name} - {e}")

        highly_correlated_vars[group_name] = highly_correlated_pairs  # Assign outside the try block

    return highly_correlated_vars


# Assuming your DataFrames are named df_FRED_Desc_MD and df_FRED_MD
highly_correlated_vars = select_highly_correlated_variables(
    df_FRED_Desc_MD, df_FRED_MD, 'Group', 'FRED', corr_threshold=0.8
)

# Print the highly correlated variables for each group
for group_name, pairs in highly_correlated_vars.items():
    print(f"\nGroup: {group_name}")
    if pairs:
        for var1, var2, corr_value in pairs:
            print(f"  - {var1} and {var2} (Correlation: {corr_value:.2f})")
    else:
        print("  - No highly correlated variables found.")


In [ ]:
# Get the list of series in df_FRED_Desc_MD
listed_series = set(df_FRED_Desc_MD['FRED'])

# Get the list of columns in df_FRED_MD
all_columns = set(df_FRED_MD.columns)

# Find columns in df_FRED_MD that are not listed in df_FRED_Desc_MD
unmatched_columns = all_columns - listed_series
# Display the result
print("Columns in df_FRED_MD that do not belong to any group:")
print(unmatched_columns)

In [ ]:

def correlation_filter(df_desc, df_data, group_column, series_column, target_variable, corr_threshold=0.6):
    """
    Selects at most two variables per group based on absolute correlation with the target variable.

    Args:
      df_desc: DataFrame containing series descriptions and group names.
      df_data: DataFrame containing the time series data.
      group_column: Name of the column with group labels in df_desc.
      series_column: Name of the column with series names in df_desc.
      target_variable: The dependent variable for correlation analysis.
      corr_threshold: Minimum absolute correlation value for selection.

    Returns:
      Dictionary with group names as keys and selected variables as values.
    """
    selected_vars = {}
    group_names = df_desc[group_column].unique()

    for group_name in group_names:
        try:
            # Get series for the current group
            series_names = df_desc[df_desc[group_column] == group_name][series_column]
            group_data = df_data[series_names].copy()

            # Drop missing values
            group_data.dropna(inplace=True)

            # Compute correlation with target variable
            corr_values = group_data.corrwith(df_data[target_variable]).abs().sort_values(ascending=False)

            # Select the top two variables that exceed the correlation threshold
            selected_vars[group_name] = list(corr_values[corr_values >= corr_threshold].index[:2])

        except KeyError as e:
            print(f"Warning: Series not found in group {group_name} - {e}")
            selected_vars[group_name] = []

    return selected_vars


def rfe_feature_selection(df_desc, df_data, group_column, series_column, target_variable):
    """
    Selects at most two variables per group using Recursive Feature Elimination (RFE).

    Args:
      df_desc: DataFrame containing series descriptions and group names.
      df_data: DataFrame containing the time series data.
      group_column: Name of the column with group labels in df_desc.
      series_column: Name of the column with series names in df_desc.
      target_variable: The dependent variable for feature selection.

    Returns:
      Dictionary with group names as keys and selected variables as values.
    """
    selected_vars = {}
    group_names = df_desc[group_column].unique()

    for group_name in group_names:
        try:
            # Get series for the current group
            series_names = df_desc[df_desc[group_column] == group_name][series_column]
            group_data = df_data[series_names].copy()
            group_data.dropna(inplace=True)  # Remove missing values

            # Prepare features (X) and target (y)
            X = group_data
            y = df_data.loc[X.index, target_variable]

            # Apply RFE with Linear Regression
            model = LinearRegression()
            selector = RFE(model, n_features_to_select=min(2, X.shape[1]))  # At most 2 features
            selector.fit(X, y)

            # Get selected features
            selected_vars[group_name] = list(X.columns[selector.support_])

        except KeyError as e:
            print(f"Warning: Series not found in group {group_name} - {e}")
            selected_vars[group_name] = []
        except ValueError as e:
            print(f"Skipping group {group_name} due to insufficient data: {e}")
            selected_vars[group_name] = []

    return selected_vars


def mutual_info_feature_selection(df_desc, df_data, group_column, series_column, target_variable):
    """
    Selects at most two variables per group using Mutual Information.

    Args:
      df_desc: DataFrame containing series descriptions and group names.
      df_data: DataFrame containing the time series data.
      group_column: Name of the column with group labels in df_desc.
      series_column: Name of the column with series names in df_desc.
      target_variable: The dependent variable for feature selection.

    Returns:
      Dictionary with group names as keys and selected variables as values.
    """
    selected_vars = {}
    group_names = df_desc[group_column].unique()

    for group_name in group_names:
        try:
            # Get series for the current group
            series_names = df_desc[df_desc[group_column] == group_name][series_column]
            group_data = df_data[series_names].copy()
            group_data.dropna(inplace=True)  # Remove missing values

            # Prepare features (X) and target (y)
            X = group_data
            y = df_data.loc[X.index, target_variable]

            # Compute Mutual Information scores
            mi_scores = mutual_info_regression(X, y)
            mi_df = pd.DataFrame({"Feature": X.columns, "MI_Score": mi_scores})
            mi_df = mi_df.sort_values("MI_Score", ascending=False)

            # Select at most two features with the highest MI score
            selected_vars[group_name] = list(mi_df["Feature"][:2])

        except KeyError as e:
            print(f"Warning: Series not found in group {group_name} - {e}")
            selected_vars[group_name] = []
        except ValueError as e:
            print(f"Skipping group {group_name} due to insufficient data: {e}")
            selected_vars[group_name] = []

    return selected_vars


# ==============================
# Example Usage
# ==============================

# Assuming df_FRED_Desc_MD contains the group descriptions
# df_FRED_MD contains the time series data
# The target variable is 'CPIAUCSL'

target_var = "CPIAUCSL"

# Run Correlation Filter
corr_selected = correlation_filter(df_FRED_Desc_MD, df_FRED_MD, "Group", "FRED", target_var, corr_threshold=0.6)

# Run Recursive Feature Elimination
rfe_selected = rfe_feature_selection(df_FRED_Desc_MD, df_FRED_MD, "Group", "FRED", target_var)

# Run Mutual Information Selection
mi_selected = mutual_info_feature_selection(df_FRED_Desc_MD, df_FRED_MD, "Group", "FRED", target_var)

# Print results
print("\n--- Correlation Filter Selected Variables ---")
for group, variables in corr_selected.items():
    print(f"Group: {group}, Selected: {variables}")

print("\n--- RFE Selected Variables ---")
for group, variables in rfe_selected.items():
    print(f"Group: {group}, Selected: {variables}")

print("\n--- Mutual Information Selected Variables ---")
for group, variables in mi_selected.items():
    print(f"Group: {group}, Selected: {variables}")


In [ ]:
print(df_FRED_MD.head())
print(df_FRED_MD.shape)

df_FRED_MD_t = df_FRED_MD.copy()
df_FRED_MD_t = df_FRED_MD_t[['CPIAUCSL', 'UNRATE', 'HOUST', 'HOUSTNE', 'PERMITNE', 'AAA', 'BAA', 'PERMITMW', 'RETAILx', 'TB3MS', 'TB6MS', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]

column_lengths = df_FRED_MD_t.count()
print(column_lengths)

print(df_FRED_MD_t['S&P 500'])


In [ ]:

qty_series = df_FRED_MD.shape[1]


for i in range(0,qty_series):
    str_transf = df_FRED_Desc_MD.iloc[i,2]
    str_ticker = df_FRED_Desc_MD.iloc[i,3]
    col_ticker = np.where(df_FRED_MD_t.columns == str_ticker)
    if len(col_ticker[0]) > 0:
        col_ticker = col_ticker[0][0]
        df_series = df_FRED_MD_t.iloc[:,col_ticker]
        if str_transf == "First difference of log":
            df_FRED_MD_t.iloc[:,col_ticker] = np.log(df_series).diff()
        elif str_transf == "First difference":
            df_FRED_MD_t.iloc[:,col_ticker] = df_series.diff()
        elif str_transf == "Log":
            df_FRED_MD_t.iloc[:,col_ticker] = np.log(df_series)
        elif str_transf == "Second difference of log":
            df_FRED_MD_t.iloc[:,col_ticker] = np.log(df_series).diff().diff()
        elif str_transf == "Second difference":
            df_FRED_MD_t.iloc[:,col_ticker] = df_series.diff().diff()
        elif str_transf == "First difference of (ratio - 1)":
            df_FRED_MD_t.iloc[:,col_ticker] = df_series.pct_change().diff()

df_series = df_FRED_MD['S&P 500']
# df_FRED_MD_t['S&P 500'] = np.log(df_series).diff()
# df_FRED_MD_t = df_FRED_MD_t.iloc[2:,:] # Removes the first 2 rows due to differencing
# print(df_FRED_MD_t.head())
# print(df_FRED_MD_t.tail())


# print(df_FRED_MD_t.head())
# print(df_FRED_MD_t.tail())
print(df_FRED_MD_t['S&P 500'])
print(df_FRED_MD_t.shape)


In [ ]:
df_FRED_MD_t[['CPIAUCSL', 'UNRATE', 'HOUST', 'HOUSTNE', 'AAA', 'BAA', 'RETAILx', 'TB3MS', 'TB6MS', 'FEDFUNDS', 'CP3Mx', 'S&P 500']]
df_FRED_MD_t.dropna(inplace=True)

print(df_FRED_MD_t['S&P 500'].head())
print(df_FRED_MD_t.shape)

In [ ]:
import matplotlib.dates as mdates

def test_stationarity_adf(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'  {key}: {value}')

# Example usage
# test_stationarity_adf(df_FRED_MD['S&P 500'])  # Replace 'column_name' with your actual column name
# print(df_date)

def plot_series(series, df_date=None, title='Time Series'):
    plt.figure(figsize=(12, 6))
    if df_date is not None:
        # plt.plot(df_date, series, label='Time Series')
        plt.plot(df_date, series, label='Time Series')
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Customize date format
    else:
        plt.plot(series, label='Time Series')
    plt.title(title)
    plt.xlabel('Date' if df_date is not None else 'Time')
    plt.ylabel('Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels by 45 degrees
    plt.legend()
    plt.tight_layout()       # Adjust layout to prevent clipping
    plt.show()

def plot_rolling_statistics(series, df_date=None, title='Time Series'):
    rolling_mean = series.rolling(window=12).mean()
    rolling_std = series.rolling(window=12).std()
    plt.figure(figsize=(12, 6))
    # plt.plot(series, label='Original Series')
    if df_date is not None:
        plt.plot(df_date, series, label='Time Series')
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    else:
        plt.plot(series, label='Original Series')

    plt.plot(rolling_mean, color='red', label='Rolling Mean')
    plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.title(title)
    plt.legend()
    plt.show()

def difference(series):
    return series.diff().dropna()

# Example usage
# plot_series(df_FRED_MD['S&P 500'], df_date, title='Original S&P 500 Time Series')  # Replace 'column_name' with your actual column name
# plot_rolling_statistics(df_FRED_MD['S&P 500'], df_date, title='Original S&P 500 Time Series: Rolling Stats')  # Replace 'column_name' with your actual column name

# Difference the series
# differenced_series = difference(df_FRED_MD['S&P 500'])

# Plot the differenced series
params = ['CPIAUCSL', 'UNRATE', 'HOUST', 'AAA', 'BAA', 'RETAILx', 'TB3MS', 'TB6MS', 'FEDFUNDS', 'CP3Mx', 'S&P 500']
for i in params:
    test_stationarity_adf(df_FRED_MD_t[i])
    plot_series(df_FRED_MD_t[i], title=('Differenced Time Series' + i))
    # plot_rolling_statistics(df_FRED_MD_t[i], title=('Differenced Time Series: Rolling Stats' + i))




In [ ]:


# Data transformation according to McCraken and Ng (2016)
# Monthly database

# Normalization (mean = 0 and std = 1)
df_FRED_MD_t_norm = pd.DataFrame(data = scale(df_FRED_MD_t),
                                 index = df_FRED_MD_t.index,
                                 columns = df_FRED_MD_t.columns)
df_FRED_MD_t_norm.head()

# Data preparation using normalized data

# Exclude nan from the transformed time series
df_FRED_MD_t_norm_ex_nan = df_FRED_MD_t_norm.dropna()

# Dataframe containing inflation time series only
# CPIAUCSL - Consumer Price Index for All Urban Consumers: All Items
y = df_FRED_MD_t_norm_ex_nan["CPIAUCSL"]
X = df_FRED_MD_t_norm_ex_nan.drop(columns = "CPIAUCSL")

print("OTHER DATA")
print(df_FRED_MD_t_norm.head())
print(df_FRED_MD_t_norm.tail())

# print(y.tail())


# scaler_other = StandardScaler()
# scaled_all = df_FRED_MD_t.drop(columns = "CPIAUCSL")
# scaled_all_features = scaler_other.fit_transform(scaled_all)  
# df_FRED_MD_t_norm2 = pd.DataFrame(data=scaled_all_features, index=scaled_all.index, columns=scaled_all.columns)
# 
# list = ['CPIAUCSL', 'UNRATE', 'HOUST', 'RETAILx', 'FEDFUNDS', 'CP3Mx', 'S&P 500']
# for i in list:
#     plot_series(df_FRED_MD_t_norm_ex_nan[i], title=('Differenced Time Series' + i))
#     plot_rolling_statistics(df_FRED_MD_t_norm_ex_nan[i], title=('Differenced Time Series: Rolling Stats' + i))


# Print the first few rows of both DataFrames to compare 
# print("First few rows using scale method:\n", other_data.head()) 
# print("\nFirst few rows using StandardScaler:\n", df_FRED_MD_t_norm2.head())

# print(other_data.shape)
# print(df_FRED_MD_t_norm_ex_nan.index)

In [ ]:

index_ref = df_FRED_MD_t_norm_ex_nan.index

for rnd_state in range(0,10):
    index_train, index_test = train_test_split(index_ref, test_size = 0.2, random_state = rnd_state)

    X.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + 'X.csv')
    y.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + 'y.csv')

    # Split samples

    y_train, y_test = y.loc[index_train], y.loc[index_test]
    X_train, X_test = X.loc[index_train], X.loc[index_test]

    # Normalization

    y_train = pd.DataFrame(data = scale(y_train), index = y_train.index)
    y_test = pd.DataFrame(data = scale(y_test), index = y_test.index)

    X_train = pd.DataFrame(data = scale(X_train), index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(data = scale(X_test), index = X_test.index, columns = X_test.columns)

    # Save raw database (split)

    y_train.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + 'y_train.csv')
    y_test.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + 'y_test.csv')
    X_train.to_csv(str_Dir_Plan_PC + str(rnd_state) + ' ' + 'X_train.csv')
    X_test.to_csv(str_Dir_Plan_PC+ str(rnd_state) + ' ' + 'X_test.csv')

    #####################################################################################################################
    #                                                                                                                   #
    # Denoising and Compression                                                                                         #
    #                                                                                                                   #
    #####################################################################################################################

    
    # Variational autoencoder

    X_train_pca, X_test_pca = pca_decomp(X_train, X_test, threshold = 0.9, plot_name = 'X_full_pca')

    # Reads dimensions from PCA
    n = X_train.shape[1]
    print("N", n)

    n_pca = X_train_pca.shape[1]
    print("N_PCA", n_pca)

    X_train_vae, X_test_vae = vae(X_train, X_test,
                                  intermediate_dim = int((n + n_pca)/2), latent_dim = n_pca,
                                  batch_size = 16, epochs = 100,
                                  verbose = False, plot_name = 'X_full_vae')

    X_train_ae, X_test_ae = deep_ae(X_train, X_test,
                                    intermediate_dim = int((n + n_pca)/2), latent_dim = n_pca,
                                    batch_size = 16, epochs = 100,
                                    verbose = False, plot_name = 'X_full_ae')
    

    
